# Ollama on Google Colab with Google Drive + Public Access

Run Ollama AI models in Google Colab with persistent storage via Google Drive and public web access via ngrok.

## 🚀 Quick Start

1. **Mount Google Drive** (for persistent model storage)
2. **Install Ollama** in the Colab VM
3. **Configure model directory** to use Drive
4. **Download models** (stored in Drive for reuse)
5. **Start Ollama server**
6. **Expose publicly** with ngrok tunnel

## ⚠️ Limitations

- **Session timeout**: ~90 min idle, ~12 hours max (free Colab)
- **Resource constraints**: Limited RAM/CPU - big models may not fit
- **Not production-safe**: Unpredictable uptime and latency
- **Treat as demo/sandbox**: Great for testing, not production use

## 📋 Prerequisites

- Google account with Colab access
- Google Drive space for models (GGUF files can be large)
- ngrok account (free tier available)

---

## 1. Mount Google Drive

Mount your Google Drive to store Ollama models persistently. This prevents re-downloading large model files every session.

In [ ]:
# Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

# Create Ollama directory in Drive (if it doesn't exist)
import os

ollama_drive_path = "/content/drive/MyDrive/ollama"
os.makedirs(ollama_drive_path, exist_ok=True)
os.makedirs(f"{ollama_drive_path}/models", exist_ok=True)

print(f"✅ Google Drive mounted and Ollama directory created at: {ollama_drive_path}")
print(f"📁 Models will be stored in: {ollama_drive_path}/models")


## 2. Install Ollama

Install Ollama in the Colab environment. We'll use the official installation script.

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

# Verify installation
!ollama --version

print("✅ Ollama installed successfully!")


## 3. Configure Ollama to Use Google Drive

Set up Ollama to use your Google Drive directory for model storage instead of the default Colab location.

In [ ]:
# Set environment variables to use Drive for Ollama data
import os

ollama_drive_path = "/content/drive/MyDrive/ollama"

# Set OLLAMA_MODELS to point to Drive
os.environ["OLLAMA_MODELS"] = f"{ollama_drive_path}/models"

# Create symlink from default location to Drive (for compatibility)
default_models_path = "/root/.ollama/models"
os.makedirs("/root/.ollama", exist_ok=True)

# Remove existing symlink/directory if it exists
if os.path.exists(default_models_path):
    if os.path.islink(default_models_path):
        os.unlink(default_models_path)
    else:
        import shutil

        shutil.rmtree(default_models_path)

# Create symlink to Drive
os.symlink(f"{ollama_drive_path}/models", default_models_path)

print(f"✅ Ollama configured to use Google Drive: {ollama_drive_path}/models")
print(f"📁 Symlink created: {default_models_path} -> {ollama_drive_path}/models")


## 4. Download Models

Download your desired models. Since they're stored in Google Drive, they'll persist across sessions.

**Popular lightweight models for Colab:**
- `qwen2.5:3b` - Fast, capable 3B parameter model
- `deepseek-coder:1.3b` - Code-focused model
- `gemma:2b` - Google's lightweight model
- `phi3:3.8b` - Microsoft's efficient model

**Note:** Large models (>7B) may not fit in Colab's RAM/CPU constraints.

In [ ]:
# Choose your models (uncomment the ones you want)
models_to_download = [
    "qwen2.5:3b",  # Fast, capable general model
    "deepseek-coder:1.3b",  # Code-focused model
    "gemma:2b",  # Google's lightweight model
    # "phi3:3.8b",       # Microsoft's efficient model (uncomment if needed)
    # "llama3:8b",       # Large model - may not fit in Colab (uncomment if you have Pro)
]

print("🚀 Starting model downloads...")
print(f"📁 Models will be saved to Google Drive: {ollama_drive_path}/models")
print()

for model in models_to_download:
    print(f"⬇️ Downloading {model}...")
    !ollama pull {model}
    print(f"✅ {model} downloaded successfully!")
    print()

print("🎉 All models downloaded!")
print("📋 Available models:")
!ollama list


## 5. Start Ollama Server

Start the Ollama server in the background. It will run on localhost:11434.

In [ ]:
# Start Ollama server in background
import subprocess
import time

print("🚀 Starting Ollama server...")

# Start server in background
process = subprocess.Popen(
    ["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

# Wait a moment for server to start
time.sleep(3)

# Check if server is running
try:
    result = subprocess.run(
        ["curl", "-s", "http://localhost:11434/api/tags"],
        capture_output=True,
        text=True,
        timeout=5,
    )
    if result.returncode == 0:
        print("✅ Ollama server started successfully!")
        print("🌐 Server running on: http://localhost:11434")
    else:
        print("❌ Server may not be responding yet, but process started")
except:
    print(
        "⚠️ Server process started, but health check failed (this is normal initially)"
    )

# Test with a simple model
print("\n🧪 Testing with a simple model...")
!ollama run qwen2.5:3b "Hello!" --format json


## 6. Set Up Public Access with ngrok

Expose your Ollama server to the internet using ngrok. This gives you a public URL that forwards to your Colab instance.

**Requirements:**
- Free ngrok account (sign up at ngrok.com)
- ngrok auth token

In [ ]:
# Install ngrok
!pip install pyngrok

from pyngrok import ngrok
import os

# Set your ngrok auth token (replace with your actual token)
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE"  # ⚠️ Replace with your token!

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTH_TOKEN_HERE":
    print("❌ Please set your ngrok auth token!")
    print("1. Sign up at https://ngrok.com")
    print("2. Get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken")
    print("3. Replace YOUR_NGROK_AUTH_TOKEN_HERE with your actual token")
else:
    # Authenticate ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    
    # Start tunnel to Ollama port
    print("🌐 Starting ngrok tunnel to Ollama server...")
    tunnel = ngrok.connect(11434, "http")
    
    public_url = tunnel.public_url
    print(f"✅ Public URL: {public_url}")
    print(f"🌍 Your Ollama API is now accessible at: {public_url}/v1/chat/completions")
    print()
    print("📋 Example API call:")
    print(f"curl -X POST {public_url}/v1/chat/completions \")
    print("  -H 'Content-Type: application/json' \")
    print("  -d '{\"model\": \"qwen2.5:3b\", \"messages\": [{\"role\": \"user\", \"content\": \"Hello!\"}]}'")
    
    # Keep tunnel alive
    print("\n🔄 Tunnel is active. Keep this cell running to maintain the connection.")
    print("⚠️ Colab sessions timeout, so this URL will only work while the session is active.")


## 7. Alternative: Pinggy Tunnel (if ngrok doesn't work)

If ngrok has issues, you can use Pinggy as an alternative tunneling service.

In [ ]:
# Alternative: Pinggy tunnel
# Download and install Pinggy
!wget https://pinggy.io/cli/pinggy_0.1.1_linux_amd64.tar.gz
!tar -xzf pinggy_0.1.1_linux_amd64.tar.gz
!chmod +x pinggy

print("🌐 Starting Pinggy tunnel...")
print("📋 Your public URL will be displayed below:")
print("⚠️ Keep this cell running to maintain the tunnel")
print()

# Start Pinggy tunnel (this will run indefinitely)
!./pinggy -p 11434 http


## 8. Test Your Public API

Test that your public Ollama API is working correctly.

In [ ]:
# Test the public API
import requests
import json

# Get the public URL (replace with your actual ngrok URL)
# If using ngrok, copy the URL from the previous cell
# If using Pinggy, copy the URL from the Pinggy output

public_url = "YOUR_PUBLIC_URL_HERE"  # Replace with your actual URL

if public_url == "YOUR_PUBLIC_URL_HERE":
    print("❌ Please set your public URL!")
    print("Copy the URL from the ngrok or Pinggy output above.")
else:
    try:
        # Test models endpoint
        response = requests.get(f"{public_url}/api/tags", timeout=10)
        if response.status_code == 200:
            models = response.json()
            print("✅ Public API is working!")
            print("📋 Available models:")
            for model in models.get("models", []):
                print(f"  • {model['name']}")
        else:
            print(f"❌ API returned status code: {response.status_code}")

        # Test chat completion
        print("\n🧪 Testing chat completion...")
        payload = {
            "model": "qwen2.5:3b",
            "messages": [{"role": "user", "content": "Say hello in 10 words or less."}],
            "stream": False,
        }
        response = requests.post(
            f"{public_url}/v1/chat/completions", json=payload, timeout=30
        )
        if response.status_code == 200:
            result = response.json()
            content = result["choices"][0]["message"]["content"]
            print(f"🤖 Response: {content}")
            print("✅ Chat completion working!")
        else:
            print(f"❌ Chat completion failed: {response.status_code}")

    except Exception as e:
        print(f"❌ Test failed: {e}")
        print("💡 Make sure:")
        print("   1. Ollama server is running")
        print("   2. The tunnel is active")
        print("   3. You copied the correct public URL")


## 9. Integration with Goblin Assistant

Configure your Goblin Assistant backend to use this Colab-hosted Ollama instance.

In [ ]:
# Configuration for Goblin Assistant
public_url = "YOUR_PUBLIC_URL_HERE"  # Replace with your actual URL

if public_url != "YOUR_PUBLIC_URL_HERE":
    print("🔧 Goblin Assistant Configuration:")
    print("=" * 50)
    print()
    print("# Automatic setup using the integration script:")
    print(f"cd /path/to/ForgeMonorepo")
    print(f"python3 setup_colab_ollama_integration.py --colab-url {public_url} --auto-test")
    print()
    print("# Or manually update providers.toml with:")
    print("[providers.ollama_colab]")
    print('name = "Ollama (Colab)"')
    print(f'endpoint = "{public_url}"')
    print('capabilities = ["chat", "reasoning", "code", "embedding"]')
    print('models = ["qwen2.5:3b", "deepseek-coder:1.3b", "gemma:2b"]')
    print("priority_tier = 0")
    print("cost_score = 0.0")
    print("default_timeout_ms = 30000")
    print("bandwidth_score = 0.5")
    print("supports_cot = false")
    print()
    print("⚠️ Remember: Colab sessions timeout, so this endpoint is temporary!")
    print("🔄 Use the setup script to update the URL when you restart Colab.")
else:
    print("❌ Please set your public URL first!")


## 📚 Troubleshooting

### Common Issues:

1. **Colab session crashed/restarted**
   - Models in Drive persist, but you need to remount Drive and restart Ollama
   - Run cells 1-5 again to get back up and running

2. **Out of RAM/CPU**
   - Use smaller models (3B or less parameters)
   - Upgrade to Colab Pro for more resources
   - Close other Colab tabs

3. **ngrok/Pinggy connection issues**
   - Check your auth token is correct
   - Try the alternative tunneling service
   - Make sure Ollama is running on port 11434

4. **Models not persisting**
   - Ensure Drive is mounted correctly
   - Check that the symlink was created
   - Verify models are in `/content/drive/MyDrive/ollama/models`

### Performance Tips:

- Use lightweight models for best performance
- Keep Colab tab focused to prevent timeouts
- Use Colab Pro for longer sessions and more resources
- Test with simple prompts first

---

## 🎉 Success!

You now have Ollama running in Colab with:
- ✅ Persistent model storage via Google Drive
- ✅ Public web access via ngrok/Pinggy
- ✅ Integration ready for Goblin Assistant
- ✅ Cost-effective AI inference (free Colab)

**Remember:** This is perfect for demos, testing, and development, but not production use due to session timeouts!

**Next steps:**
1. Copy your public URL
2. Configure your Goblin Assistant backend
3. Start chatting with your AI models! 🤖